# Desafio

### Acesse a api da camara, e recupere os reembolsos do ano de 2018

In [1]:
import requests

# fazendo o download do conteúdo do arquivo
url = "http://www.camara.leg.br/cotas/Ano-2018.csv.zip"  # pega o arquivo .zip e na variavel r
r = requests.get(url)   

In [2]:
r.content
r.status_code

200

In [3]:
with open("Ano-2018.csv.zip", "wb") as arquivo:  # cria um novo arquivo para escrita em binario
    arquivo.write(r.content)                     # persiste o conteudo de r neste arquivo .zip


In [4]:
from zipfile import ZipFile
# descompacto todo o conteudo de .zip para nova variavel
zipado = ZipFile("Ano-2018.csv.zip", "r")
# listo os arquivos descompactados   
zipado.namelist()                           

['Ano-2018.csv']

In [5]:
# extraio um dos arquivos para nova pasta
zipado.extract(member="Ano-2018.csv", path="reembolso-2018") 

'reembolso-2018\\Ano-2018.csv'

In [6]:
zipado.close()

### Leia o Arquivo csv 

In [7]:
import pandas as pd
pd.set_option('display.max_columns', None)
DTYPE = {
    'txNomeParlamentar': str,
    'cpf' : str,
    'ideCadastro': str,
    'nuCarteiraParlamentar': str,
    'nuLegislatura': str,
    'sgUF': str,
    'sgPartido': str,
    'codLegislatura': str,
    'numSubCota': str,
    'txtDescricao': str,
    'numEspecificacaoSubCota': str,
    'txtDescricaoEspecificacao': str,
    'txtFornecedor': str,
    'txtCNPJCPF': str,
    'txtNumero': str,
    'indTipoDocumento': str,
    'datEmissao': str,
    'vlrDocumento': float,
    'vlrGlosa': str,
    'vlrLiquido': float,
    'numMes': str,
    'numAno': str,
    'numParcela': str,
    'txtPassageiro': str,
    'txtTrecho': str,
    'numLote': str,
    'numRessarcimento': str,
    'nuDeputadoId': str,
    'ideDocumento': str
}

df_ano_2018 = pd.read_csv("reembolso-2018/Ano-2018.csv", sep=';', dtype=DTYPE)
df_ano_2018.shape

(292632, 31)

### Visualize 10 linhas aleatórias

In [8]:
df_ano_2018.sample(10)

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,numEspecificacaoSubCota,txtDescricaoEspecificacao,txtFornecedor,txtCNPJCPF,txtNumero,indTipoDocumento,datEmissao,vlrDocumento,vlrGlosa,vlrLiquido,numMes,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
10175,OSMAR SERRAGLIO,01773852949,73463,463,2015,PR,PP,55,122,"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",0,NaN,CENTRO DE GESTÃO DE MEIOS DE PGTO. LTDA. - SEM...,040.882.080/0016-5,1820523702,0,2018-06-28T00:00:00,333.78,0,333.78,6,2018,0,NaN,NaN,1513112,NaN,NaN,1439,6627283,https://www.camara.leg.br/cota-parlamentar/doc...
161501,Dimas Fabiano,03694339639,160599,231,2015,MG,PP,55,122,"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",0,NaN,Autopista Fernão Dias,093.263.420/0017-0,001063,1,2018-07-26T00:00:00,2.30,0,2.30,7,2018,0,NaN,NaN,1523047,NaN,NaN,2356,6652420,https://www.camara.leg.br/cota-parlamentar/doc...
224218,Aluisio Mendes,66746485749,178881,68,2015,MA,PODE,55,14,"HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO ...",0,NaN,LA HOTELS EMPREENDIMENTOS 1 LTDA - (GT BRASILI...,091.095.670/0173-2,1151721,4,2018-06-22T00:00:00,1391.50,0,1391.50,6,2018,0,NaN,NaN,1508714,NaN,NaN,2999,6615127,http://camara.leg.br/cota-parlamentar/nota-fis...
72717,JAIME MARTINS,17614899687,74665,240,2015,MG,PROS,55,10,TELEFONIA,0,NaN,TELEFONICA BRASIL S.A.,025.581.570/0091-0,0242826535,0,2018-12-13T00:00:00,226.68,0,226.68,12,2018,0,NaN,NaN,1551443,NaN,NaN,1050,6730326,NaN
162323,Arthur Oliveira Maia,32604637553,160600,192,2015,BA,DEM,55,10,TELEFONIA,0,NaN,RAMAL,000.000.000/0000-6,6727723,0,NaN,81.24,0,81.24,11,2018,0,NaN,NaN,0,0,NaN,2225,0,NaN
201521,Dr. Sinval Malheiros,88779254853,178830,554,2015,SP,PODE,55,3,COMBUSTÍVEIS E LUBRIFICANTES.,1,Veículos Automotores,A.P.CENTRAL DE CATANDUVA LTDA,153.230.590/0019-8,5357,4,2018-07-05T00:00:00,5000.00,0,5000.00,7,2018,0,NaN,NaN,1513101,NaN,NaN,2887,6627328,http://camara.leg.br/cota-parlamentar/nota-fis...
135946,Luis Tibé,81460910672,160510,251,2015,MG,AVANTE,55,3,COMBUSTÍVEIS E LUBRIFICANTES.,1,Veículos Automotores,POSTO RIEGERT LTDA,166.314.180/0013-7,769075,0,2018-08-03T00:00:00,70.00,0,70.00,8,2018,0,NaN,NaN,1522616,NaN,NaN,2412,6652506,https://www.camara.leg.br/cota-parlamentar/doc...
4450,JÚLIA MARINHO,10799397253,67312,34,2015,PA,PSC,55,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,NaN,SERVICO NACIONAL DE APRENDIZAGEM COMERCIAL SENAC,334.691.720/0067-2,30238,4,2018-11-13T00:00:00,8.15,0,8.15,11,2018,0,NaN,NaN,1539728,NaN,NaN,3054,6700130,http://camara.leg.br/cota-parlamentar/nota-fis...
91240,André Figueiredo,25905503320,133439,89,2015,CE,PDT,55,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,0,NaN,COMPANHIA ENERGETICA DO CEARA,070.472.510/0017-0,511605384,0,2018-02-02T00:00:00,493.77,0,493.77,2,2018,0,NaN,NaN,1470749,NaN,NaN,1748,6518061,https://www.camara.leg.br/cota-parlamentar/doc...
263977,Carlos Gomes,02142743730,178962,492,2015,RS,PRB,55,3,COMBUSTÍVEIS E LUBRIFICANTES.,1,Veículos Automotores,SAO JOAO POSTOS DE ABASTECIMENTO E SERVICOS LTDA,371.304.810/0016-0,138638,4,2018-05-28T00:00:00,100.00,0,100.00,5,2018,0,NaN,NaN,1500234,NaN,NaN,2900,6593332,http://camara.leg.br/cota-parlamentar/nota-fis...


### Existem registros com `ideDocumento` nulo?

In [9]:
# df_ano_2018.info()
filtro = df_ano_2018.ideDocumento.isna()
df_ano_2018["sgPartido"] = df_ano_2018[filtro].txNomeParlamentar.replace("LIDERANÇA DO ","")

df_ano_2018.head(1)

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,numEspecificacaoSubCota,txtDescricaoEspecificacao,txtFornecedor,txtCNPJCPF,txtNumero,indTipoDocumento,datEmissao,vlrDocumento,vlrGlosa,vlrLiquido,numMes,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
0,LID.GOV-CD,NaN,NaN,NaN,2015,NaN,NaN,55,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,NaN,BISCOITOS CASEIROS HOMONNAI LTDA,046.448.200/0017-7,10473,4,2018-12-20T00:00:00,454.41,0,454.41,12,2018,0,NaN,NaN,1550817,NaN,NaN,2812,6728796,http://camara.leg.br/cota-parlamentar/nota-fis...


### Converta a coluna `datEmissao` para `datetime`, e descubra quantos reembolsos aconteram no primeiro semestre

In [10]:
df_ano_2018['datEmissao'] = pd.to_datetime(
    df_ano_2018.datEmissao,
    format='%Y-%m-%d'
)

In [11]:
df_ano_2018.head(1)

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,numEspecificacaoSubCota,txtDescricaoEspecificacao,txtFornecedor,txtCNPJCPF,txtNumero,indTipoDocumento,datEmissao,vlrDocumento,vlrGlosa,vlrLiquido,numMes,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
0,LID.GOV-CD,NaN,NaN,NaN,2015,NaN,NaN,55,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,NaN,BISCOITOS CASEIROS HOMONNAI LTDA,046.448.200/0017-7,10473,4,2018-12-20,454.41,0,454.41,12,2018,0,NaN,NaN,1550817,NaN,NaN,2812,6728796,http://camara.leg.br/cota-parlamentar/nota-fis...


In [12]:
primeiro_semestre = (df_ano_2018['datEmissao'] >= '2018-01-01') &  (df_ano_2018['datEmissao'] <= '2018-06-30')

In [27]:
df_filtrado = df_ano_2018[ primeiro_semestre].copy()
df_filtrado.datEmissao.value_counts().sum()

159594

In [13]:
df_ano_2018 = df_ano_2018.fillna(value='Não se aplica')

In [14]:
df_ano_2018.head(1)

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,numEspecificacaoSubCota,txtDescricaoEspecificacao,txtFornecedor,txtCNPJCPF,txtNumero,indTipoDocumento,datEmissao,vlrDocumento,vlrGlosa,vlrLiquido,numMes,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
0,LID.GOV-CD,Não se aplica,Não se aplica,Não se aplica,2015,Não se aplica,Não se aplica,55,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,Não se aplica,BISCOITOS CASEIROS HOMONNAI LTDA,046.448.200/0017-7,10473,4,2018-12-20 00:00:00,454.41,0,454.41,12,2018,0,Não se aplica,Não se aplica,1550817,Não se aplica,Não se aplica,2812,6728796,http://camara.leg.br/cota-parlamentar/nota-fis...


### Limpe a coluna `txtCNPJCPF`: ela deve conter somente dígitos numéricos

In [15]:
# vamos trocar qualquer caracter que não seja um dígito número por '', essa operação remove os dígitos
df_ano_2018["txtCNPJCPF"] = df_ano_2018["txtCNPJCPF"].str.replace(r'\D', '', regex=True)

df_ano_2018.txtCNPJCPF

0         04644820000177
1         10638597000158
2         24882567000105
3         24882567000105
4         03713903000108
               ...      
292627    00000000000006
292628    00982933000121
292629    27402097000115
292630    02214932000162
292631    02012862000160
Name: txtCNPJCPF, Length: 292632, dtype: object

### Quais são os valores únicos do campo `indTipoDocumento`? Substitua o conteúdo pela respectiva legenda

*Dica*: 0 (Zero), para Nota Fiscal; 1 (um), para Recibo; e 2, para Despesa no Exterior.

In [16]:
df_ano_2018.indTipoDocumento.unique()


array(['4', '0', '1', '2', '3'], dtype=object)

In [17]:
df_ano_2018.indTipoDocumento.value_counts()


0    177181
4     63975
1     51313
3       130
2        33
Name: indTipoDocumento, dtype: int64